Вам нужно защитить данные клиентов страховой компании «Хоть потоп». Разработайте такой метод преобразования данных, чтобы по ним было сложно восстановить персональную информацию. Обоснуйте корректность его работы.

Нужно защитить данные, чтобы при преобразовании качество моделей машинного обучения не ухудшилось. Подбирать наилучшую модель не требуется.

## 1. Загрузка данных

#### Добавим необходимые библиотеки

In [62]:
import pandas as pd
import numpy as np

#### Импортируем данные

In [63]:
insurance_df = pd.read_csv('D:/3D Objects/Praktikum/Data/insurance.csv')
print(insurance_df.info())
insurance_df.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5000 entries, 0 to 4999
Data columns (total 5 columns):
Пол                  5000 non-null int64
Возраст              5000 non-null float64
Зарплата             5000 non-null float64
Члены семьи          5000 non-null int64
Страховые выплаты    5000 non-null int64
dtypes: float64(2), int64(3)
memory usage: 195.4 KB
None


,Пол,Возраст,Зарплата,Члены семьи,Страховые выплаты
0,1,41.0,49600.0,1,0
1,0,46.0,38000.0,1,1
2,0,29.0,21000.0,0,0
3,0,21.0,41700.0,2,0
4,1,28.0,26100.0,0,0


В датасете содержатся следующие данные о клиентах страховой компании:
- Пол
- Возраст
- Зарплата
- Члены семьи
- Страховые выплаты

Всего **5000** записей, пропусков нет.

## Задача

Предложить алгоритм преобразования данных для сокрытия персональной информации о клиентах,\
при этом не ухудшив качество моделей машинного обучения.

**Признаки** - пол, возраст и зарплата застрахованного, количество членов его семьи.\
**Целевой признак** - количество страховых выплат клиенту.

---

## 2. Умножение матриц

Для ответа на вопрос обучим модель линейной регрессии на исходной матрицы, затем обучим модель на матрице, полученной в результате умножения на обратимую.

В качестве метрики используем **r2 score**

In [64]:
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score

In [65]:
train_features, test_features, train_target, test_target = train_test_split(
insurance_df.drop('Страховые выплаты', axis=1),
insurance_df['Страховые выплаты'],
test_size=0.2,
random_state=42
)

In [66]:
# проведем масштабирование признаков, указав normalize=True
model = LinearRegression(normalize=True)
model.fit(train_features, train_target)
r2_score(test_target, model.predict(test_features))

0.4368694923138

Получаем **r2_score = 0.437**\
Посмотрим, изменится ли значение при умножении признаков на обратимую матрицу.

Генерируем матрицу и проверяем ее на обратимость:

In [67]:
P = np.random.normal(loc=0, scale=1, size=(4,4))
print(np.linalg.inv(P))
P

[[ 0.46526824 -0.60320263  0.79771227 -0.76777569]
 [-0.42643733 -0.83453547  0.40931236 -1.03839264]
 [-0.23318507 -0.6057911   0.20754258  0.37176567]
 [-1.02763547 -0.3061004   0.374154   -0.22004795]]


array([[ 0.37894039, -0.06743203,  0.12022943, -0.80083966],
       [ 0.39087821, -0.9001693 , -1.21765584,  0.82681866],
       [ 1.41151158, -1.3620091 , -0.15551521,  1.23955128],
       [ 0.08662722, -0.74876148,  0.86793024,  0.15298846]])

In [68]:
train_features_modified = train_features @ P
test_features_modified = test_features @ P

In [69]:
model.fit(train_features_modified, train_target)
r2_score(test_target, model.predict(test_features_modified))

0.4368694923125649

Видим, что после умножения признаков на обратимую матрицу метрика не изменилась.

---

Обозначения:

- $X$ — матрица признаков (нулевой столбец состоит из единиц)

- $y$ — вектор целевого признака

- $P$ — матрица, на которую умножаются признаки

- $w$ — вектор весов линейной регрессии (нулевой элемент равен сдвигу)

Предсказания:

$$
a = Xw
$$

Задача обучения:

$$
w = \arg\min_w MSE(Xw, y)
$$

Формула обучения:

$$
w = (X^T X)^{-1} X^T y
$$

**Ответ:** Качество линейной регрессии не изменится.

**Обоснование:** при умножении признаков на обратимую матрицу задача обучения примет следующий вид:

$$
w = \arg\min_w MSE(XPw, y)
$$

Формула весов изменится на:
$$
w = (XP^T XP)^{-1} XP^T y
$$

Преобразуем:
$$
(XP^T XP)^{-1} XP^T y = (XP)^{-1} (XP^T)^{-1} XP^T y = (XP)^{-1} Ey = P^{-1} X^{-1} y
$$

Получаем:
$$
a = XPw = XP P^{-1} X^{-1} y = X X^{-1} y = y
$$

## 3. Алгоритм преобразования

**Алгоритм:**

1. Вычисляется количество столбцов в матрице признаков
2. Создается квадратная обратимая матрица соответствующего размера
3. Признаки умножаются на эту матрицу
4. Модель снова обучается
5. Делаются предсказания

Математически доказано выше, что при таком подходе качество предсказаний не изменится.

## 4. Проверка алгоритма

#### Разделяем данные на выборки:

In [70]:
train_features, test_features, train_target, test_target = train_test_split(
insurance_df.drop('Страховые выплаты', axis=1),
insurance_df['Страховые выплаты'],
test_size=0.2,
random_state=42
)

#### Функция, умножающая признаки на обратимую матрицу:

In [71]:
def encode_df(train_features, test_features):
    
    if len(train_features.columns) != len(test_features.columns):
        return print('ERROR')
    
    num_columns = len(train_features.columns)
    matrix = np.random.normal(loc=0, scale=1, size=(num_columns, num_columns))
    
    # на случай, если сгенерировалась необратимая матрица
    try:
        np.linalg.inv(matrix)
    except:
        matrix = np.random.normal(loc=0, scale=1, size=(num_columns, num_columns))
        
    return train_features @ matrix, test_features @ matrix

#### Сохраняем зашифрованные признаки:

In [72]:
train_features_modified, test_features_modified = encode_df(train_features, test_features)

#### Строим модель на признаках до преобразования:

In [73]:
model_before = LinearRegression(normalize=True)
model_before.fit(train_features, train_target)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None, normalize=True)

#### На признаках после преобразования:

In [74]:
model_after = LinearRegression(normalize=True)
model_after.fit(train_features_modified, train_target)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None, normalize=True)

#### Сравниваем:

In [75]:
print('r2 score before:', r2_score(test_target, model_before.predict(test_features)))
print('r2 score after:', r2_score(test_target, model_after.predict(test_features_modified)))

r2 score before: 0.4368694923138
r2 score after: 0.4368694923137525


## Вывод:

### Значение метрики r2 score на двух моделях одинаково, алгоритм работает.

---

## Чек-лист проверки

Поставьте 'x' в выполненных пунктах. Далее нажмите Shift+Enter.

- [x]  Jupyter Notebook открыт
- [x]  Весь код выполняется без ошибок
- [x]  Ячейки с кодом расположены в порядке исполнения
- [x]  Выполнен шаг 1: данные загружены
- [x]  Выполнен шаг 2: получен ответ на вопрос об умножении матриц
    - [x]  Указан правильный вариант ответа
    - [x]  Вариант обоснован
- [x]  Выполнен шаг 3: предложен алгоритм преобразования
    - [x]  Алгоритм описан
    - [x]  Алгоритм обоснован
- [x]  Выполнен шаг 4: алгоритм проверен
    - [x]  Алгоритм реализован
    - [x]  Проведено сравнение качества моделей до и после преобразования